# M51 - MGE

Closed $2\times 2$ economy, monopoly X producer MPSGE version

|Markets  |   X  |    Y  |      W    |  CONS |  ENTRE  |
|---------|------|-------|-----------|-------|---------|
|    PX   |  100 |       |   -100    |       |         | 
|    PY   |      |  100  |   -100    |       |         | 
|    PU   |      |       |    200    |  -180 |    -20  |   
|    PW   |  -32 |  -60  |           |    92 |         |  
|    PZ   |  -48 |  -40  |           |    88 |         |   
|    MK   |  -20 |       |           |       |     20  |   

In [1]:
using MPSGE_MP

In [15]:
M51 = MPSGEModel()

σ = 9


@sectors(M51, begin
	X,	(description = "Activity level for sector X",)
	Y,	(description = "Activity level for sector Y",)
	W,	(description = "Activity level for sector W (Hicksian welfare index)",)
end)
@commodities(M51, begin
	PX,	(description = "Price index for commodity X",)
	PY,	(description = "Price index for commodity Y",)
	PW,	(description = "Price index for primary factor L (net of tax)",)
	PZ,	(description = "Price index for primary factor K",)
	PU,	(description = "Price index for welfare (expenditure function)",)
end)
@consumers(M51, begin
	CONS,	(description = "Representative agent",)
	ENTRE,	(description = "Entreprenuer (monopolist)",)
end)
@auxiliaries(M51, begin
	SHAREX,	(description = "Value share of good X",)
	MARKUP,	(description = "X sector markup on marginal cost",)
end)

@production(M51, X, [s=1,t=0], begin
    @output(PX, 80, t, taxes = [Tax(ENTRE, MARKUP)])
    @input(PW, 32, s)
    @input(PZ, 48, s)
end)

@production(M51, Y, [s=1,t=0], begin
    @output(PY, 100, t)
    @input(PW, 60, s)
    @input(PZ, 40, s)
end)

@production(M51, W, [s=σ,t=0], begin
    @output(PU, 200, t)
    @input(PX, 80, s, reference_price=1.25)
    @input(PY, 100, s)
end)

@demand(M51, CONS, begin
    @final_demand(PU, 180)
end,begin
    @endowment(PW, 92)
    @endowment(PZ, 88)
end)

@demand(M51, ENTRE, begin
    @final_demand(PU, 20)
end,begin
end)

@aux_constraint(M51, SHAREX, 
    SHAREX - (80*PX*X/ (80*PX*X + 100*PY*Y))
)

@aux_constraint(M51, MARKUP,
    MARKUP - 1/(σ - (σ - 1)*SHAREX)
);

# Benchmark

In [16]:
fix(PU, 1)

set_start_value(PX, 1.25)
set_start_value(SHAREX, .5)
set_start_value(MARKUP, .2)

solve!(M51, cumulative_iteration_limit=0)
generate_report(M51)


Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_FCA2.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 11

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,1.0,0.0
2,W,1.0,0.0
3,Y,1.0,0.0
4,PU,1.0,0.0
5,PW,1.0,0.0
6,PZ,1.0,0.0
7,PY,1.0,0.0
8,PX,1.25,0.0
9,ENTRE,20.0,0.0


In [21]:
incomeM = value(W*ENTRE/(ENTRE+CONS))
incomeC = value(W*CONS/(ENTRE+CONS))

println("incomeM: $incomeM\nincomeC: $incomeC")

incomeM: 0.1
incomeC: 0.9


# Counterfactual

In [22]:
fix(MARKUP,0)

solve!(M51)
generate_report(M51)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_F084.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 10

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             4.0746e+01             0.0e+00 (zero_profit[X)
    1     1     1    10 1.1170e+01  1.0e+00    0.0e+00 (zero_profit[W)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.1170e+01           I 0.0e+00 1.1e+01 (zero_profit[W)
    1     1     3     3 7.9897e-03  1.0e+00 SO 0.0e+00 5.8e-03 (market_clearanc)
    2     1     4     4 3.2216e-09  1.0e+00 SO 0.0e+00 2.2e-09 (market_clearanc)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Functi

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,1.7449,-7.52074e-10
2,W,1.03973,-3.93442e-10
3,Y,0.387179,-8.89059e-10
4,PU,1.0,-4.13112e-9
5,PW,0.998044,1.08673e-9
6,PZ,1.31961,4.33445e-10
7,PY,1.11601,2.15392e-9
8,PX,1.18012,-1.69038e-9
9,ENTRE,0.0,0.0


In [23]:
incomeM = value(W*ENTRE/(ENTRE+CONS))
incomeC = value(W*CONS/(ENTRE+CONS))

println("incomeM: $incomeM\nincomeC: $incomeC")

incomeM: 0.0
incomeC: 1.0397267822532776
